In [2]:
import numpy as np
from matplotlib import pyplot as plt

For the cosserat rod equations, there are two kinematic equations (one related to translations and another related to rotations) and two dynamic equations (again, one related to translations and another related to rotations). So we tackle this problem in increasing levels of complexity via these milestones:

1) first tackle translations (which are easier to implement) by testing the equations on an elastic beam that is fixed on one end and has a small axial force on the one end. Because the beam is elastic, and the axial force is small, the entire beam behaves like a spring. We can then plot look at how much the beam stretches, which should correspond with analytical spring equations. This is milestone 1.

In [3]:
# My simple cosserat rod

n_elements = 20


"""
nodal quantities
"""
n_nodes = n_elements + 1
forces = np.zeros((3, n_nodes))
velocities = np.zeros((3, n_nodes))

needs_to_be_initialized = []

masses = np.zeros((1, n_nodes)) # needs to be initialized
needs_to_be_initialized.append("masses")
positions = np.ones((3, n_nodes)) # needs to be initialized
needs_to_be_initialized.append("positions")

"""
element quantities
"""
lengths_bold = positions[:, 1:] - positions[:, :-1]
print(lengths_bold)
lengths_norm = np.linalg.norm(lengths_bold, 
                              axis=0, 
                              keepdims=True)

# reference_lengths_bold = lengths_bold.copy() 
reference_lengths_bold =  np.ones((3, n_elements))
reference_lengths_norm = np.linalg.norm(reference_lengths_bold, 
                                        axis=0, 
                                        keepdims=True)
tangents = lengths_bold / lengths_norm
dilatations = lengths_norm / reference_lengths_norm

# 
# directors = [np.eye(3) for _ in range(n_elements)]
# directors = np.array([np.eye(3) for _ in range(n_elements)])
# 
directors = np.zeros((3, 3, n_elements))
for idx in range(n_elements):
    directors[:, :, idx] = np.eye(3)
needs_to_be_initialized.append("directors")
    
radius = np.ones((1, n_elements)) 
needs_to_be_initialized.append("radius")

areas = np.pi * (radius**2)
# Oracle gave it to us
G = 20
E = 40
shear_stiffness_matrix = np.zeros((3, 3, n_elements))
alpha_c = 4.0 / 3.0
shear_stiffness_matrix[0, 0, :] = alpha_c * G * areas
shear_stiffness_matrix[1, 1, :] = alpha_c * G * areas
shear_stiffness_matrix[2, 2, :] = E * areas

needs_to_be_initialized.append("shear_stiffness_matrix")

# sigma
shear_stretch_strains = dilatations * tangents - directors[2, :, :]

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


/tmp/ipykernel_1645/2192802790.py:34: RuntimeWarning: invalid value encountered in true_divide
  tangents = lengths_bold / lengths_norm


In [26]:
needs_to_be_initialized

['masses', 'positions', 'directors', 'radius', 'shear_stiffness_matrix']

In [30]:
# Modified trapezoidal integration
def modified_diff(t_x):
    """ Modified trapezoidal integration"""
    # Pads a 0 at the end of an array
    temp = np.pad(t_x, (0,1), 
                  'constant', 
                  constant_values=(0,0)) # Using roll calculate the diff (ghost node of 0)
    return (temp - np.roll(temp, 1))

In [ ]:
shear_stretch_strains = dilatations * tangents - directors[2, :, :]

# dv_dt
(modified_diff(shear_stiffness_matrix @ shear_stretch_strains 
               / dilatations) + forces) / masses 

In [ ]:
# (3, n_nodes)
# x x x x x .... 
# y y y y y ....
# z z z z z ....

# x x x x x .... x y y y y y .... y z z z z z ....z 


# (n_nodes, 3)
# x y z
# x y z
# ...
# x y z
# x y z x y z x yz ........


In [11]:
def position_verlet(dt, x, v, a):
    """Does one iteration/timestep using the Position verlet scheme
    
    Parameters
    ----------
    dt : float
        Simulation timestep in seconds
    x : float/array-like
        Quantity of interest / position of COM
    v : float/array-like
        Quantity of interest / velocity of COM
    force_rule : ufunc
        A function, f, that takes one argument and
        returns the instantaneous forcing

    Returns
    -------
    x_n : float/array-like
        The quantity of interest at the Next time step
    v_n : float/array-like
        The quantity of interest at the Next time step
    """
    temp_x = x + 0.5*dt*v
    v_n = v + dt * force_rule(temp_x)
    x_n = temp_x + 0.5 * dt * v_n
    return x_n, v_n

a.timestep_using(position_verlet)

Variable                  Type        Data/Info
-----------------------------------------------
black_reformat            function    <function black_reformat at 0x103062310>
dilatations               ndarray     20: 20 elems, type `float64`, 160 bytes
directors                 ndarray     3x3x20: 180 elems, type `float64`, 1440 bytes
forces                    ndarray     3x21: 63 elems, type `float64`, 504 bytes
idx                       int         19
json                      module      <module 'json' from '/usr<...>hon3.9/json/__init__.py'>
lengths_bold              ndarray     3x20: 60 elems, type `float64`, 480 bytes
lengths_norm              ndarray     20: 20 elems, type `float64`, 160 bytes
masses                    ndarray     1x21: 21 elems, type `float64`, 168 bytes
n_elements                int         20
n_nodes                   int         21
needs_to_be_initialized   list        n=4
np                        module      <module 'numpy' from '/us<...>kages/numpy/__init